<h1 style="text-align: center;" markdown="1">Reinforcement Learning: Algoritmo Monte Carlo</h1>

Il metodo Monte Carlo per il **reinforcement learning** apprende direttamente da episodi di esperienza senza alcuna conoscenza preliminare delle transizioni. Qui, il componente casuale è il ritorno o la ricompensa.

Un avvertimento è che questo metodo può essere applicato solo a problemi episodici. Il motivo è che l'episodio deve terminare prima di poter calcolare eventuali ritorni. Qui, non eseguiamo un aggiornamento dopo ogni azione, ma piuttosto dopo ogni episodio. Utilizza l'idea più semplice: il valore è il ritorno medio di tutte le traiettorie di esempio per ciascuno stato.

Analogamente alla programmazione dinamica, vi è una valutazione regola (individuazione della funzione valore per una determinata regola casuale) e una fase di miglioramento della regola (ricerca della regola ottimale).

### Valutazione della regola (policy evaluation)
L'obiettivo qui è quello di imparare la funzione di valore vpi(s) da episodi di esperienza con una regola pi. Il rendimento è la ricompensa totale scontata. Possiamo stimare qualsiasi valore atteso semplicemente sommando i campioni e dividendo per il numero totale di campioni. 

La domanda è: come otteniamo questi ritorni campione? Per questo, abbiamo bisogno di generare ed eseguire un sacco di episodi. Per ogni episodio che eseguiamo, avremo una sequenza di stati e ricompense. E da queste ricompense, possiamo calcolare il ritorno che è solo la somma di tutti i premi futuri. 

Ci sono due tipi di algoritmi per policy evaluation:
* **First visit Monte Carlo**: si calcola il redimento medio solo la prima volta che si vede l'episodio.
* **Every visit Monte Carlo**: Si calcola il rendimento medio per ogni volta che viene visitato in un episodio.

### Miglioramento regola (policy improvement)
Una volta che abbiamo la funzione valore per una regola casuale, l'importante compito che rimane rimane quello di trovare la regola ottimale usando Monte Carlo. 
Il miglioramento delle regole viene fatto rendendo la regola avida rispetto alla funzione del valore corrente. In questo caso, abbiamo una funzione valore-azione e quindi non è necessario alcun modello per costruire la politica avida. Una regola avida favorirà sempre una certa azione se la maggior parte delle azioni non viene esplorata correttamente. Ci sono due soluzioni per questo:
* **Monte Carlo con exploring starts**: Tutte le coppie di azioni e stato hanno probabilità diversa da zero di essere la coppia di partenza. Questo garantirà che ogni episodio riprodotto porterà l'agente in nuovi stati e quindi, c'è più esplorazione dell'ambiente.
* **Monte Carlo con epsilon-Soft**: Cosa succede se esiste un unico punto di partenza per un ambiente (ad esempio, una partita a scacchi)? L'esplorazione degli inizi non è l'opzione giusta in questi casi. L'idea più semplice per garantire l'esplorazione continua. Tutte le azioni sono provate con probabilità diversa da zero (1 - epsilon) e viene scelta l'azione che massimizza la funzione del valore d'azione e poi con probabilità epsilon si sceglie un'azione a caso.

Per lo sviluppo dell'algoritmo Monte Carlo ho tradotto lo pseudo codice, che si può trovare in qualsiasi sito internet, in python e applico questo algoritmo ad il caso discusso precedentemente.

In [2]:
import random
import time
import numpy as np
import gym

In [3]:
# Tecnica Montecarlo
class Montecarlo:
    
    def __init__(self, environment, grid_delta):
        '''
        arguments:
        grid_delta - passo per la discretizzazione delle osservazioni
        '''
        self._env = gym.make(environment)
        self._grid_delta = grid_delta
        self.q_dict = {}
        
    def run_episode(self, epsilon, sleep=None):
        trajectory = []
        o = self._env.reset()
        s = self._get_state(o)
        trajectory.append(s)
        done = False
        n = 0
        while not done:
            a = self._choose_action(s , epsilon)
            o, r, done, _ = self._env.step(a)
            #Per vedere quando impara: Attenzione il processo diventa molto più lento
            #self._env.render()
            s = self._get_state(o)
            trajectory += [a,r,s]
            n += 1
        self.update_q(trajectory)
        return n
    
    def learn(self, num_episodes, epsilon0, decay,  sleep=None):
        counters = [] # lista con le durate degli episodi
        epsilon = epsilon0
        for i in range(num_episodes):
            counters.append(self.run_episode(epsilon))
            if (i+1) % 100 == 0:
                print(f"epsilon={epsilon:.2f}, lunghezza media={sum(counters[-100:])/100}")
                epsilon = epsilon * decay
        self._env.close()
        
    def _perform(self, sleep=None):
        observation = self._env.reset()
        state = self._get_state(observation)
        self._env.render()
        done = False
        count = 0
            
#        while not done:
        for _ in range(200):
            if not done:
                count = count + 1
            action = self._choose_action(state, 0)
            observation, reward, done, info = self._env.step(action)
            state = self._get_state(observation)
            self._env.render()
            if sleep:
                time.sleep(sleep)
                
        self._env.close()
        print(count)
        
    def perform(self, n, sleep=None):
        for _ in range(n):
            self._perform(sleep)
            
    def close(self):
        self._env.close()
        
    def reset_env(self):
        self._env.reset()
        
    def _get_state(self, observation):
        return tuple([value // self._grid_delta for value in observation.tolist()])
    
    def _choose_action(self, s, epsilon):
        if random.random() < epsilon:
            return self._env.action_space.sample()
        else:
            return self.get_best_action(s)
        
    def get_best_action(self, s):
        action_values = self.q_dict.get(s, None)
        if action_values is None:
            return self._env.action_space.sample()
        i = np.argmax([tuple_[1] for tuple_ in action_values])
        return action_values[i][0]
    
    def update_q(self, trajectory):
        while len(trajectory) > 3:
            s, a = trajectory[:2]
            rewards = sum(trajectory[2::3])
            self.update_q_s_a(s, a, rewards)
            trajectory = trajectory[3:]
            
    def update_q_s_a(self, s, a, rewards):
        found = False
        s_list = self.q_dict.get(s, [])
        for index, terna in enumerate(s_list):
            if terna[0] == a:
                found = True
                s_list[index] = (a,  ((terna[1]*terna[2] + rewards) / (terna[2]+1)), terna[2]+1)
        if not found:
            s_list.append([a, rewards, 1])
        self.q_dict[s] = s_list


Una volta che abbiamo l'algoritmo Monte Carlo pronto possiamo utilizzarlo per diversi ambienti. Come primo esempio vi mostro l'ambiente gym **CartPole-v0** che è quello che simula il video youtube visto in precedenza.

In [ ]:
mc = Montecarlo(environment='CartPole-v0', grid_delta=0.5)

In [ ]:
mc.perform(n=5, sleep=0.01)

In [ ]:
mc.learn(num_episodes=2_000, epsilon0=0.6, decay=0.7,  sleep = None)

In [ ]:
mc.perform(n=10, sleep=0.01)

Come secondo ambiente vi mostro il **Pendulum-v0**. Il problema dell'oscillazione del pendolo invertito è un problema classico nella letteratura di controllo. In questa versione del problema, il pendolo inizia in una posizione casuale e l'obiettivo è di farlo ruotare in modo che rimanga dritto.

In [4]:
mc1 = Montecarlo(environment='Pendulum-v0', grid_delta=0.5)

/Users/giuliomori/gymenv/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
mc1.perform(n=5, sleep=0.01)

200
200
200
200
200


In [6]:
mc1.learn(num_episodes=2_000, epsilon0=0.6, decay=0.7,  sleep = None)

epsilon=0.60, lunghezza media=200.0
epsilon=0.42, lunghezza media=200.0
epsilon=0.29, lunghezza media=200.0
epsilon=0.21, lunghezza media=200.0
epsilon=0.14, lunghezza media=200.0
epsilon=0.10, lunghezza media=200.0
epsilon=0.07, lunghezza media=200.0
epsilon=0.05, lunghezza media=200.0
epsilon=0.03, lunghezza media=200.0
epsilon=0.02, lunghezza media=200.0
epsilon=0.02, lunghezza media=200.0
epsilon=0.01, lunghezza media=200.0
epsilon=0.01, lunghezza media=200.0
epsilon=0.01, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0
epsilon=0.00, lunghezza media=200.0


In [7]:
mc1.perform(n=10, sleep=0.01)

200
200
200
200
200
200
200
200
200
200


Ma non per tutti gli ambienti il metodo montecarlo è efficace. Possiamo prendere l'esempio di **Acrobot-v1**, Il sistema di acrobot comprende due giunti e due collegamenti, in cui viene attivata la giunzione tra i due collegamenti. Inizialmente, i collegamenti sono sospesi verso il basso e l'obiettivo è di far oscillare la fine del collegamento inferiore fino a una determinata altezza.

In [8]:
mc2 = Montecarlo(environment='Acrobot-v1', grid_delta=0.5)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [9]:
mc2.perform(n=5, sleep=0.01)

200
200
200
200
200


In [ ]:
mc2.learn(num_episodes=2_000, epsilon0=0.6, decay=0.7,  sleep = None)

In [ ]:
mc2.perform(n=10, sleep=0.01)